In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [10]:
indep.shape

(400, 3)

In [11]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [12]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [13]:
from sklearn.neighbors import KNeighborsClassifier

In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'metric': ['minkowski', 'euclidean', 'manhattan'],
    'p': [1, 2]  # Only relevant if metric='minkowski'
} 

grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['minkowski', 'euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 7, 9], 'p': [1, 2]},
             scoring='f1', verbose=3)

In [15]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [16]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'metric': 'minkowski', 'n_neighbors': 3, 'p': 1}: 0.7681406676150807


In [17]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[70 15]
 [16 33]]


In [18]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.81      0.82      0.82        85
           1       0.69      0.67      0.68        49

    accuracy                           0.77       134
   macro avg       0.75      0.75      0.75       134
weighted avg       0.77      0.77      0.77       134



In [19]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

np.float64(0.8116446578631453)

In [20]:
table=pd.DataFrame.from_dict(re)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_n_neighbors,param_p,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006514,0.000870,0.008615,0.001287,minkowski,3,1,"{'metric': 'minkowski', 'n_neighbors': 3, 'p': 1}",0.756757,0.727273,0.628571,0.717949,0.648649,0.695840,0.048867,1
1,0.005000,0.000655,0.008549,0.000794,minkowski,3,2,"{'metric': 'minkowski', 'n_neighbors': 3, 'p': 2}",0.756757,0.727273,0.628571,0.717949,0.648649,0.695840,0.048867,1
2,0.004495,0.001557,0.008278,0.001378,minkowski,5,1,"{'metric': 'minkowski', 'n_neighbors': 5, 'p': 1}",0.648649,0.628571,0.545455,0.800000,0.717949,0.668125,0.085866,13
3,0.003842,0.000456,0.006771,0.001879,minkowski,5,2,"{'metric': 'minkowski', 'n_neighbors': 5, 'p': 2}",0.648649,0.628571,0.545455,0.800000,0.717949,0.668125,0.085866,13
4,0.003621,0.000481,0.007023,0.000810,minkowski,7,1,"{'metric': 'minkowski', 'n_neighbors': 7, 'p': 1}",0.628571,0.736842,0.606061,0.750000,0.611111,0.666517,0.063371,19
5,0.003742,0.000445,0.007948,0.001523,minkowski,7,2,"{'metric': 'minkowski', 'n_neighbors': 7, 'p': 2}",0.628571,0.736842,0.606061,0.750000,0.611111,0.666517,0.063371,19
6,0.004329,0.001326,0.007040,0.001136,minkowski,9,1,"{'metric': 'minkowski', 'n_neighbors': 9, 'p': 1}",0.666667,0.702703,0.685714,0.800000,0.611111,0.693239,0.061636,7
7,0.004047,0.001219,0.008521,0.000818,minkowski,9,2,"{'metric': 'minkowski', 'n_neighbors': 9, 'p': 2}",0.666667,0.702703,0.685714,0.800000,0.611111,0.693239,0.061636,7
8,0.003323,0.000464,0.008256,0.002397,euclidean,3,1,"{'metric': 'euclidean', 'n_neighbors': 3, 'p': 1}",0.756757,0.727273,0.628571,0.717949,0.648649,0.695840,0.048867,1
9,0.003641,0.001210,0.008723,0.000589,euclidean,3,2,"{'metric': 'euclidean', 'n_neighbors': 3, 'p': 2}",0.756757,0.727273,0.628571,0.717949,0.648649,0.695840,0.048867,1


In [22]:
age_input=float(input("Age:"))
bmi_input=float(input("EstimatedSalary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age: 31
EstimatedSalary: 150000
Sex Male 0 or 1: 1


In [23]:
Future_Prediction=grid.predict([[age_input,bmi_input,sex_male_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


C:\Users\Priyan\anaconda3\envs\aiml\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
